In [121]:
from typing import Annotated, Literal, TypedDict, List

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

from pydantic import BaseModel


from dotenv import load_dotenv

from DataLayer import DataLayer

import os
import random

In [8]:
load_dotenv()

True

In [142]:
dl = DataLayer()

Successfully connected to the database. 12 users found.


In [153]:
dir(dl)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_channel',
 'add_channel_membership',
 'add_chunks',
 'add_message',
 'add_thread',
 'add_user',
 'conn_string',
 'create_channel',
 'get_associated_files',
 'get_channel',
 'get_channel_by_id',
 'get_channel_by_name',
 'get_channel_messages',
 'get_channels_for_user',
 'get_file',
 'get_message',
 'get_messages_by_user',
 'get_messages_in_channel',
 'get_my_channels',
 'get_recent_messages',
 'get_user',
 'get_user_by_username',
 'get_user_status',
 'get_users_in_channel',
 'join_channel',
 'pool',
 'save_file',
 'search_channels',
 'search_messages',
 'send_message',
 'similarity_search'

In [152]:
dl.get_messages_for_users(["davidrag"])

AttributeError: 'DataLayer' object has no attribute 'get_messages_for_users'

In [143]:
# Define the tools for the agent to use
@tool
def get_messages_for_users(usernames: List[str]):
    """Get all messages from specified users within an optional time frame."""
    return dl.get_messages_for_users(usernames)
    


tools = [get_messages_for_users]
tool_node = ToolNode(tools)


In [144]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY2"),
).bind_tools(tools)


In [145]:


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}



In [146]:
def should_continue(state: MessagesState) -> Literal["tools", END]:
    last_message = state['messages'][-1]
    print(last_message)
    if last_message.tool_calls:
        return "tools"
    
    return END


In [147]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_edge("tools", "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)

checkpointer = MemorySaver()

app = workflow.compile(checkpointer=checkpointer)




In [148]:
checkpointer = MemorySaver()

In [149]:
app = workflow.compile(checkpointer=checkpointer)

In [150]:
app.invoke(
    {"messages": [HumanMessage(content="Hey, can you check the messages for the user 'davidrag' and tell me what he's working on?")]},
    config={"configurable": {"thread_id": 42}}
)


content='' additional_kwargs={'tool_calls': [{'id': 'call_wUbr1wXhkGkdbX981sd42xL4', 'function': {'arguments': '{"usernames":["davidrag"]}', 'name': 'get_messages_for_users'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 75, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0c468d3a-f866-462e-bb30-db8345ce4241-0' tool_calls=[{'name': 'get_messages_for_users', 'args': {'usernames': ['davidrag']}, 'id': 'call_wUbr1wXhkGkdbX981sd42xL4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 75, 'output_tokens': 21, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'

{'messages': [HumanMessage(content="Hey, can you check the messages for the user 'davidrag' and tell me what he's working on?", additional_kwargs={}, response_metadata={}, id='41b3f6ce-c9ff-4288-98f8-6b85bbad1b8e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wUbr1wXhkGkdbX981sd42xL4', 'function': {'arguments': '{"usernames":["davidrag"]}', 'name': 'get_messages_for_users'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 75, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c468d3a-f866-462e-bb30-db8345ce4241-0', tool_calls=[{'name': 'get_messages_for_users', 'args': {'usernames': [